In [1]:
import yaml
import os
from datetime import datetime as dt
import pandas as pd
import numpy as np
import boto3
from sqlalchemy import create_engine

In [2]:
with open('config.yml', 'r') as file:
    config = yaml.safe_load(file)

In [3]:
# Establish connection to AWS environment
aws_creds = config['awsaccount']

os.environ["AWS_ACCESS_KEY_ID"] = aws_creds['accesskey']
os.environ["AWS_SECRET_ACCESS_KEY"] = aws_creds['secretkey']
os.environ["AWS_REGION"] = "us-east-1"

s3 = boto3.resource('s3')
client = boto3.client('s3')

bucket_name = "stocks-daily-ohlc"
bucket = s3.Bucket(bucket_name)

In [4]:
# Get list of bucket objects
for obj in bucket.objects.all():
    print(obj.key)

AAPL.csv
MSFT.csv


In [5]:
obj_list = pd.DataFrame(client.list_objects_v2(Bucket = bucket_name)['Contents'])
obj_list

,Key,LastModified,ETag,Size,StorageClass
0,AAPL.csv,2023-09-14 20:34:33+00:00,"""09bc7cb21266b175b453100a5dd6c038""",5355,STANDARD
1,MSFT.csv,2023-09-14 18:29:24+00:00,"""6c2ee92b7e80710b88189c114e470979""",5529,STANDARD


In [6]:
obj = client.get_object(Bucket = bucket_name, Key = obj_list['Key'][1])
df = pd.read_csv(obj['Body'])
df

,ticker,obs_date,open,high,low,close,volume
0,MSFT,2023-09-13,331.310,336.85,331.1700,336.06,16544412
1,MSFT,2023-09-12,335.815,336.79,331.4800,331.77,17565482
2,MSFT,2023-09-11,337.240,338.42,335.4300,337.94,16583324
3,MSFT,2023-09-08,330.090,336.16,329.4600,334.27,19548165
4,MSFT,2023-09-07,331.290,333.08,329.0300,329.91,18380995
...,...,...,...,...,...,...,...
95,MSFT,2023-04-27,295.970,305.20,295.2500,304.83,46462638
96,MSFT,2023-04-26,296.700,299.57,292.7300,295.37,64481117
97,MSFT,2023-04-25,279.510,281.60,275.3700,275.42,45772236
98,MSFT,2023-04-24,282.090,284.95,278.7200,281.77,26611014


In [7]:
db_creds = config['postgres']

In [8]:
cnx = "postgresql://{user}:{pw}@{host}:{port}/{database}".format(
        user = db_creds['username'],
        pw = db_creds['password'],
        host = db_creds['host'],
        port = db_creds['port'],
        database = db_creds['dbname'])

In [9]:
engine = create_engine(cnx)
con = engine.connect()
con

In [10]:
tbl = pd.read_sql("select * from stocks", con)
tbl.head()

,ticker,obs_date,open,high,low,close,volume
0,AAPL,2023-09-13,176.47,177.300,173.98,174.21,84090357
1,AAPL,2023-09-12,179.49,180.130,174.82,176.30,90370192
2,AAPL,2023-09-11,180.07,180.300,177.34,179.36,58953052
3,AAPL,2023-09-08,178.35,180.239,177.79,178.18,65602066
4,AAPL,2023-09-07,175.18,178.210,173.54,177.56,112488803


In [11]:
tbl.shape

(100, 7)

In [17]:
len(df)

100

In [13]:
# Append dataframe to table
df.to_sql("stocks", con, if_exists = 'append', index = False)

In [14]:
tbl = pd.read_sql("select * from stocks", con)
tbl.tail()

,ticker,obs_date,open,high,low,close,volume
195,MSFT,2023-04-27,295.97,305.20,295.2500,304.83,46462638
196,MSFT,2023-04-26,296.70,299.57,292.7300,295.37,64481117
197,MSFT,2023-04-25,279.51,281.60,275.3700,275.42,45772236
198,MSFT,2023-04-24,282.09,284.95,278.7200,281.77,26611014
199,MSFT,2023-04-21,285.01,286.27,283.0608,285.76,21676387


In [15]:
tbl.shape

(200, 7)

In [18]:
con.close()